In [51]:
import random
import subprocess

def process_video(input_file, output_file, background_audio="./background.mp3"):
    """
    使用 FFmpeg 对视频进行进一步处理以规避重复内容审核：
    1. 移除元数据。
    2. 调整分辨率到 1280x720。
    3. 动态亮度、对比度调整，随机噪声。
    4. 动态水印和遮罩。
    5. 随机剪辑与重组。
    6. 混入背景音和调整音频。
    7. 时长和播放速度随机调整。
    """
    # 随机生成参数
    watermark_x = random.randint(0, 1080)  # 水印随机 X 位置
    watermark_y = random.randint(0, 580)  # 水印随机 Y 位置
    watermark_opacity = random.uniform(0.1, 0.3)  # 水印透明度

    brightness = random.uniform(-0.05, 0.05)  # 随机亮度
    contrast = random.uniform(0.9, 1.1)       # 随机对比度
    saturation = random.uniform(0.9, 1.1)     # 随机饱和度

    # 随机剪辑时长
    start_time = random.randint(0, 5)  # 从视频的 0-5 秒内开始
    clip_duration = random.randint(30, 120)  # 剪辑长度 30 到 120 秒

    # 动态播放速度
    speed_factor = random.uniform(0.95, 1.05)  # 随机播放速度（0.95x 到 1.05x）

    # 创建 FFmpeg 命令
    command = [
        "ffmpeg", "-i", input_file,
        # 移除元数据
        "-map_metadata", "-1",
        # 视频滤镜：调整分辨率、亮度对比、随机噪声、动态水印、动态遮罩
        "-vf", (
            f"scale=1280:720,"
            f"eq=brightness={brightness}:contrast={contrast}:saturation={saturation},"
            f"noise=alls=10:allf=t+u,"
            f"drawtext=text='Watermark':fontcolor=white@{watermark_opacity}:fontsize=24:"
            f"x={watermark_x}:y={watermark_y}"
        ),
        # 随机剪辑
        "-ss", f"00:00:{start_time:02d}",
        "-t", f"00:02:{clip_duration:02d}",
        # 设置播放速度
        "-filter:v", f"setpts={1/speed_factor}*PTS",
        # 音频滤镜：混入背景音、调整速度和音量
        "-filter_complex", f"[0:a]asetrate=44100*{speed_factor},volume=1.1[aud1];"
                           f"amovie={background_audio}:loop=0,volume=0.3[bg];"
                           f"[aud1][bg]amix=inputs=2:duration=shortest",
        # 视频和音频编码
        "-c:v", "libx264", "-crf", "18", "-preset", "slow",
        "-c:a", "aac",
        # 输出文件
        output_file
    ]

    # 执行 FFmpeg 命令
    try:
        subprocess.run(command, check=True)
        print(f"视频处理完成，保存为: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"处理视频时出错: {e}")

In [ ]:
import glob 

background_audio = "./background.mp3"

# 找到所有 .webm 文件并逐个处理
webm_files = glob.glob("./downloads/*.webm")
for input_file in webm_files:
    output_file = "./output.mp4"
    process_video(input_file, output_file, background_audio)